In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

In [4]:
def add_remaining_useful_life(df):
    # Get the total number of cycles for each unit
    grouped_by_unit = df.groupby(by="unit_nr")
    max_cycle = grouped_by_unit["time_cycles"].max()

    # Merge the max cycle back into the original frame
    result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='unit_nr', right_index=True)

    # Calculate remaining useful life for each row (piece-wise Linear)
    remaining_useful_life = result_frame["max_cycle"] - result_frame["time_cycles"]
    result_frame["RUL"] = remaining_useful_life

    # drop max_cycle as it's no longer needed
    result_frame = result_frame.drop("max_cycle", axis=1)

    return result_frame

In [9]:
## load data
import pandas as pd


def loading_FD001():

    # define filepath to read data
    dir_path = './CMAPSSData/'

    # define column names for easy indexing
    index_names = ['unit_nr', 'time_cycles']
    setting_names = ['setting_1', 'setting_2', 'setting_3']
    sensor_names = ['s_{}'.format(i) for i in range(1, 22)]
    col_names = index_names + setting_names + sensor_names

    # read data
    train = pd.read_csv((dir_path + 'train_FD001.txt'), sep='\s+', header=None, names=col_names)
    test = pd.read_csv((dir_path + 'test_FD001.txt'), sep='\s+', header=None, names=col_names)
    y_test = pd.read_csv((dir_path + 'RUL_FD001.txt'), sep='\s+', header=None, names=['RUL'])

    # drop non-informative features in training set
    drop_sensors = ['s_1', 's_5', 's_6', 's_10', 's_16', 's_18', 's_19']
    drop_labels = setting_names + drop_sensors
    train.drop(labels=drop_labels, axis=1, inplace=True)

    # separate title information and sensor data
    title = train.iloc[:, 0:2]
    data = train.iloc[:, 2:]

    # min-max normalization of the sensor data
    data_norm = (data - data.min()) / (data.max() - data.min())
    train_norm = pd.concat([title, data_norm], axis=1)

    # add piece-wise target remaining useful life
    train_norm = add_remaining_useful_life(train_norm)
    train_norm['RUL'].clip(upper=125, inplace=True) # in the paper the MAX RUL is mentioned as 125

    # group the training set with unit
    group = train_norm.groupby(by="unit_nr")

    # drop non-informative features in testing set
    test.drop(labels=drop_labels, axis=1, inplace=True)
    title = test.iloc[:, 0:2]
    data = test.iloc[:, 2:]
    data_norm = (data - data.min()) / (data.max() - data.min())
    test_norm = pd.concat([title, data_norm], axis=1)

    # group the testing set with unit
    group_test = test_norm.groupby(by="unit_nr")

    return group, y_test, group_test




In [10]:
group, y_test, group_test = loading_FD001()


In [11]:
group.group_keys

True

In [12]:
for i in range(0,100):
    x = group.get_group(i)

KeyError: 0

In [52]:
class CustomtrainDataset():
    def __init__(self, group,y, lookback,engine_no):
        self.group = group
        self.lookback = lookback
        self.engine_no = engine_no
        self.y = y

    def __len__(self):
        return len(self.group)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        x = self.group.get_group(self.engine_no).to_numpy()
        label = self.y[idx:idx+self.lookback]
        data_window = x[idx:idx+self.lookback]

        return data_window, label

dataset = CustomtrainDataset(group,10,i)
Dataloader = DataLoader(dataset,batch_size=1,shuffle=True)

In [53]:
j =1
x_test = group_test.get_group(j).to_numpy()
len(x_test)


31

In [ ]:
class LSTMNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=17, )